# DATA SCIENCE WEEKEND CHALLENGE

The competition will use Waze data information from Jabar Digital Service, start from September 2020 to November 2020. The data covers 3 busiest areas in West Java, Bandung, Bogor, and Bekasi. The last week of November information will be kept as a data test, meanwhile, the rest of the data will be used for training.

JDS has records of more than 7.5 million Waze reports generated by users which happen on that range and cities. There also 350k irregularities reports identified by the Waze system as irregular or atypical by taking into account historical speed data for the same day of the week and hour.

DSI and JDS invites teams to develop a model that can predict the high reports or irregularities phenomenon on certain areas, dates, and hours around the last week of November 2020. The areas are defined using S2id/S2token on level 15 which is generated with s2cell library. (its a Python library, the R library for this is here).

In [27]:
import s2cell

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [28]:
train_df = pd.read_csv('./data_train.csv')
train_df

,Ids,Labels
0,2e69e9384_2020-10-06_13,True
1,2e6992c7c_2020-10-02_17,True
2,2e69ef474_2020-09-13_19,True
3,2e69c5fd4_2020-10-10_15,True
4,2e6992134_2020-09-12_11,True
...,...,...
71331,2e69eea5c_2020-11-09_10,False
71332,2e69c5944_2020-10-27_12,True
71333,2e69f2cd4_2020-11-07_14,True
71334,2e68e64e4_2020-09-23_9,False


In [29]:
def convert_ids_to_columns(data):
    temp = train_df['Ids'].str.split("_", expand=True) 
    temp.columns = ['s2token_15', 'date', 'hour']
    
    temp['date'] = pd.to_datetime(temp['date'], format = '%Y-%m-%d')
    
    temp['Labels'] = data['Labels']
    return temp
train_df = convert_ids_to_columns(train_df)
# test_df = convert_ids_to_columns(test_df)

In [ ]:
pd.merge(train_df, alerts_df, on = 's2token_15', how = 'left')

In [3]:
ireg_df = pd.read_csv('./irregularities.csv')
ireg_df

,id,detection_date_millis,update_date_millis,street,city,is_highway,line,s2id_center,s2token_center,speed,...,delay_seconds,seconds,length,trend,type,severity,jam_level,drivers_count,alerts_count,n_thumbs_up
0,12868069,1604733149024,1604735467276,Jatiwaringin Raya,Bekasi,t,"{""line"": [{""x"": 106.91014, ""y"": -6.258107}, {""...",3344471185277583360,2e69f2d2c,13.03,...,299,432,1566,0,Small,5,3,13,0,0
1,12420463,1599906813144,1599909295834,Putri Tunggal,Depok,f,"{""line"": [{""x"": 106.887821, ""y"": -6.377016}, {...",3344462996922433536,2e69eb604,6.56,...,399,539,984,0,Small,5,3,5,0,0
2,12497533,1601728355356,1601734996933,Ir Haji Juanda,Bandung,f,"{""line"": [{""x"": 107.618629, ""y"": -6.87556}, {""...",3344176694402482176,2e68e6fc4,3.36,...,1185,1294,1212,1,Large,5,4,21,2,0
3,12536831,1602312860279,1602315706305,KH Muchtar Tabrani,Bekasi,f,"{""line"": [{""x"": 107.002934, ""y"": -6.216088}, {...",3344358143885836288,2e698c034,4.36,...,467,543,659,-1,Small,5,4,3,0,0
4,12327151,1598956623240,1598957378934,N1 Pangeran Diponegoro,Tambun Selatan,t,"{""line"": [{""x"": 107.035652, ""y"": -6.255471}, {...",3344360723013697536,2e698e5bc,4.74,...,423,474,625,0,Small,5,4,11,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352604,12362039,1599286843171,1599294437634,N8 Jalan Raya Bogor,Bogor,t,"{""line"": [{""x"": 106.815835, ""y"": -6.556333}, {...",3344419823911174144,2e69c41c4,4.57,...,677,792,1006,0,Small,5,4,12,0,0
352605,12592138,1603007434857,1603007969509,Soekarno-Hatta (Jalur Lambat),Bandung,t,"{""line"": [{""x"": 107.654988, ""y"": -6.941372}, {...",3344177896993325056,2e68e8144,4.86,...,814,915,1237,0,Large,5,4,4,1,0
352606,12823298,1604248226105,1604249587644,N1 RE Martadinata,Cikarang,t,"{""line"": [{""x"": 107.155055, ""y"": -6.257546}, {...",3344350307718004736,2e6984e2c,3.40,...,871,983,929,0,Large,5,4,2,0,0
352607,12775093,1604115788906,1604117576019,Ir Haji Juanda,Bandung,f,"{""line"": [{""x"": 107.616416, ""y"": -6.879372}, {...",3344176846873821184,2e68e71fc,9.67,...,676,913,2454,0,Small,5,3,31,0,0


In [34]:
alerts_df = pd.read_csv('./alerts.csv')
alerts_df['s2token_15']

0          2e69eeea4
1          2e69eb7f4
2          2e6994a84
3          2e699ad9c
4          2e69eec0c
             ...    
7800657    2e68e6564
7800658    2e698a284
7800659    2e68e64cc
7800660    2e69927fc
7800661    2e698c494
Name: s2token_15, Length: 7800662, dtype: object